In [13]:
import numpy as np
import os
import os.path
import pandas as pd
import cv2
import tensorflow as tf
import pickle
from PIL import Image, ImageDraw

# VARIABLES

In [4]:
model_path = '/home/deva/code/OrnithoScope/saved_models/model_efficientDet1.tflite'
input_csv_path = '/home/deva/code/OrnithoScope/data/input.csv'
columns_names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"]
output_images_directory = '/home/deva/code/data_ornitho/output_images/test_pickle/'
comparaison_images_directory = '/home/deva/code/data_ornitho/comparaison_verite_terrain/comparaison_efficientDet1/'


# FUNCTIONS

In [5]:
#@title Load the trained TFLite model and define some visualization functions

def get_classes_from_input():
    input = pd.read_csv(input_csv_path, names = columns_names)
    return input["label"].unique().tolist()

# Load the labels into a list
classes = get_classes_from_input()
# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8, results

# Génération des prédictions

In [6]:
def get_test_images():
    input = pd.read_csv(input_csv_path, names = columns_names)
    test_dataset = input.loc[input['split_value'] == 'TEST']
    list_image_test = test_dataset["file_path"].unique().tolist() # return all unique file_path to test, no prediction on two same image
    return list_image_test
loi = get_test_images()
print(loi[0])
print(len(loi))

/home/deva/code/data_ornitho/raw_data/task_2021-03-01_09/20210301-090002_(13.0).jpg
3516


In [7]:
def prediction_test_dataset():
    results = []
    list_of_images = get_test_images()
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    #list_of_images = ['/home/acarlier/code/test/2021-06-11-13-44-43.jpg']
    # Load the TFLite model
    print(len(list_of_images),' to predict')
    for img in list_of_images[:10]:
        img_name = img.split('/')[-1]
        file_path = '{}{}'.format(output_images_directory, img_name)
        if os.path.exists(file_path):
            list_of_images.remove(img)
            print(len(list_of_images),' remaining')
        else:
            INPUT_IMAGE_URL = img #@param {type:"string"}
            DETECTION_THRESHOLD = 0.3 #@param {type:"number"}
            TEMP_FILE = img
            FILE_NAME = TEMP_FILE.split("/")[-1]
            #print('test1.2 : ok')
            #!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
            #!wget -q -O $INPUT_IMAGE_URL
            #print('test1.3 : ok')
            try:
                with open(TEMP_FILE) as file:
                    im = Image.open(TEMP_FILE)
                    im.thumbnail((512, 512), Image.ANTIALIAS)
                    im.save(TEMP_FILE, 'JPEG')
                    # Run inference and draw detection result on the local copy of the original file
                    detection_result_image, result = run_odt_and_draw_results(
                        TEMP_FILE,
                        interpreter,
                        threshold=DETECTION_THRESHOLD
                    )
                    results.append(result)
                    # Show the detection result
                    #Image.fromarray(detection_result_image)
                    #result = Image.fromarray(detection_result_image)
                    #result.save("{}{}".format(output_images_directory, FILE_NAME), 'JPEG')
                    #result.show()
            except OSError as e:
                print(e.errno)
    return results

In [8]:
results = prediction_test_dataset()

3516  to predict


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2022-03-18 16:06:00.093618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 16:06:00.099374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 16:06:00.099501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-18 16:06:00.099833: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations

In [9]:
print(len(results))
print(get_classes_from_input())


10
['MESCHA', 'SITTOR', 'MESBLE', 'MESNON', 'PINARB', 'ACCMOU', 'ROUGOR', 'VEREUR', 'MOIDOM', 'TOUTUR', 'ECUROU', 'PIEBAV', 'MULGRI', 'CAMPAG', 'MESNOI', 'MESHUP']


In [14]:
pickle.dump( results, open( "/home/deva/code/OrnithoScope/saved_models/results.p", "wb" ) )


In [15]:
results

[[{'bounding_box': array([0.09921169, 0.25329775, 0.68758965, 0.4730904 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.6484375}],
 [{'bounding_box': array([0.4092438, 0.5277908, 1.0296512, 0.7281122], dtype=float32),
   'class_id': 0.0,
   'score': 0.375}],
 [{'bounding_box': array([0.5509056, 0.5129066, 0.9888495, 0.9870934], dtype=float32),
   'class_id': 0.0,
   'score': 0.9296875}],
 [{'bounding_box': array([0.39843464, 0.30333823, 0.6451436 , 0.5669474 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.84765625}],
 [],
 [{'bounding_box': array([0.41062987, 0.13668333, 0.8865938 , 0.38762027], dtype=float32),
   'class_id': 0.0,
   'score': 0.91796875}],
 [{'bounding_box': array([0.46652228, 0.13806312, 0.668348  , 0.5108628 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.8125},
  {'bounding_box': array([0.8450288, 0.8830899, 0.993529 , 0.998536 ], dtype=float32),
   'class_id': 0.0,
   'score': 0.76953125}],
 [{'bounding_box': array([0.43046698, 0.2513759 , 0.6197371 

In [46]:
def get_precision_recall_from_prediction(list_of_results, df_input_test, list_of_classes):
    class_metrics = []
    for classes in list_of_classes:
        class_metrics.append({'TP':0, 'FP':0, 'FN':0})
    
    total_correct_detections = 0
    total_incorect_detections = 0
    

    return class_metrics

In [68]:
df_input = pd.read_csv('/home/deva/code/OrnithoScope/data/input.csv', names = columns_names)
df_input_test = df_input[df_input["split_value"] == 'TEST']
df_input_test.head(1)

,split_value,file_path,label,x_min,y_min,empty_1,empty_2,x_max,y_max,empty_3
2124,TEST,/home/deva/code/data_ornitho/raw_data/task_202...,VEREUR,0.2543,0.0371,NaN,NaN,0.4593,0.7075,NaN


In [48]:
results[:3]

[[{'bounding_box': array([0.09921169, 0.25329775, 0.68758965, 0.4730904 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.6484375}],
 [{'bounding_box': array([0.4092438, 0.5277908, 1.0296512, 0.7281122], dtype=float32),
   'class_id': 0.0,
   'score': 0.375}],
 [{'bounding_box': array([0.5509056, 0.5129066, 0.9888495, 0.9870934], dtype=float32),
   'class_id': 0.0,
   'score': 0.9296875}]]

In [50]:
class_metrics = get_precision_recall_from_prediction(results,df_input_test,get_classes_from_input())

In [65]:
def print_results(class_metrics, list_of_species):
    for i in range(0, len(class_metrics)):
        TP = class_metrics[i]['TP']
        FP = class_metrics[i]['FP']
        FN = class_metrics[i]['FN']
        print(f"{list_of_species[i]} - TP : {TP} FP : {FP} FN : {FN}")

In [66]:
print_results(class_metrics, get_classes_from_input())

MESCHA - TP : 0 FP : 0 FN : 0
SITTOR - TP : 0 FP : 0 FN : 0
MESBLE - TP : 0 FP : 0 FN : 0
MESNON - TP : 0 FP : 0 FN : 0
PINARB - TP : 0 FP : 0 FN : 0
ACCMOU - TP : 0 FP : 0 FN : 0
ROUGOR - TP : 0 FP : 0 FN : 0
VEREUR - TP : 0 FP : 0 FN : 0
MOIDOM - TP : 0 FP : 0 FN : 0
TOUTUR - TP : 0 FP : 0 FN : 0
ECUROU - TP : 0 FP : 0 FN : 0
PIEBAV - TP : 0 FP : 0 FN : 0
MULGRI - TP : 0 FP : 0 FN : 0
CAMPAG - TP : 0 FP : 0 FN : 0
MESNOI - TP : 0 FP : 0 FN : 0
MESHUP - TP : 0 FP : 0 FN : 0


In [13]:
pid = os.getpid()
!kill -9 $pid